In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from dataloader import get_dataloaders
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformer import Transformer,TransformerEncoder,TransformerDecoder
import utils
import pickle
nltk.download('punkt')  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/wicaksonolxn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
BATCH_SIZE = 8 # butuh lebih banyak update 
DATA_PATH = "dataset/"  
train_loader, val_loader, test_loader = get_dataloaders(
    data_path=DATA_PATH, 
    source_lang="min", 
    target_lang="eng", 
    batch_size=BATCH_SIZE, 
    device=device
)


In [3]:
SRC_VOCAB_SIZE = 5000  
TGT_VOCAB_SIZE = 5000  
DROPOUT = 0.15      
N_LAYERS = 2         
N_HEADS = 2        
FFN_HIDDEN = 256      
D_MODEL = 128        

encoder = TransformerEncoder(SRC_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
decoder = TransformerDecoder(TGT_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
model = Transformer(encoder,decoder,device,utils.PAD_TOKEN).to(device)


In [4]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss(ignore_index=utils.PAD_TOKEN) 
print("Model initialized on:", device)

Model initialized on: cuda


In [5]:

for i, batch in enumerate(train_loader):
    if i < 1:
        print("src_batch type:", type(batch['src']))  
        print("tgt_batch type:", type(batch['tgt']))  # Should be <class 'torch.Tensor'>
        print("src_batch shape:", batch['src'].shape)  
        print("tgt_batch shape:", batch['tgt'].shape)  
    else:
        break


src_batch type: <class 'torch.Tensor'>
tgt_batch type: <class 'torch.Tensor'>
src_batch shape: torch.Size([8, 52])
tgt_batch shape: torch.Size([8, 52])


/home/wicaksonolxn/Documents/learnNLP/3.seq2seqTransformer/src/dataloader/_collate_fn.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_batch = [torch.tensor(item['src'], dtype=torch.long) for item in batch]
/home/wicaksonolxn/Documents/learnNLP/3.seq2seqTransformer/src/dataloader/_collate_fn.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_batch = [torch.tensor(item['tgt'], dtype=torch.long) for item in batch]


Checky checky .

In [8]:
pth = "dataset"
src = "min"
tgt = "eng"
tp  = os.path.join(pth,f"{src}_{tgt}")
input_dic_path = os.path.join(tp, "input_dic.pkl")
output_dic_path = os.path.join(tp, "output_dic.pkl")
with open(input_dic_path, "rb") as f:
    inp = pickle.load(f)
with open(output_dic_path, "rb") as f:
    output_dictionary = pickle.load(f)

for i,batch in enumerate(train_loader):
    if i <1:
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        output, _ = model(src_batch, tgt_batch[:, :-1]) 

        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        token_indices = torch.argmax(output, dim=1).cpu().tolist()  # Convert here
        tt = utils.detokenize(token_indices, output_dictionary)
        print(tt)
        print("Max index in output_dictionary:", max(output_dictionary.index2word.keys()))
        print("n_count:", output_dictionary.n_count)

/home/wicaksonolxn/Documents/learnNLP/3.seq2seqTransformer/src/dataloader/_collate_fn.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_batch = [torch.tensor(item['src'], dtype=torch.long) for item in batch]
/home/wicaksonolxn/Documents/learnNLP/3.seq2seqTransformer/src/dataloader/_collate_fn.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_batch = [torch.tensor(item['tgt'], dtype=torch.long) for item in batch]


KeyError: 3317


        output,_= model(src_batch, tgt_batch)
        output_dim = output.shape[-1]
        output = output[:, :-1, :].reshape(-1, output_dim)
        tgt_y  = tgt_batch[:, 1:].reshape(-1)

TypeError: detokenize() missing 1 required positional argument: 'vocab'

In [ ]:
import os
import torch
from tqdm import tqdm

EPOCHS = 250
SAVE_DIR = "saved"
os.makedirs(SAVE_DIR, exist_ok=True)
best_val_loss = float("inf")
best_model_path = None
for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")
    model.train()
    total_train_loss = 0.0
    train_bar = tqdm(train_loader, desc="🚀 Training", 
                leave=False, total=len(train_loader))
    for batch in train_loader:
        optimizer.zero_grad()
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        
        output, _ = model(src_batch, tgt_batch[:, :-1]) 
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt_y = tgt_batch[:, :-1].reshape(-1)
        
        loss = criterion(output, tgt_y)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        train_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)
    model.eval()
    total_val_loss = 0.0
    val_bar = tqdm(val_loader, desc="🚀 Validation",
              leave=True, total=len(val_loader))
    with torch.no_grad():
        for batch in val_bar:
            src_batch = batch['src'].to(device)
            tgt_batch = batch['tgt'].to(device)
            
            output, _ = model(src_batch, tgt_batch[:, :-1]) 
            output_dim = output.shape[-1]
            output = output.reshape(-1, output_dim)
            tgt_y = tgt_batch[:, :-1].reshape(-1)
            # tgt_y = tgt_batch[:, 1:].reshape(-1)

            
            loss = criterion(output, tgt_y)
            total_val_loss += loss.item()
            val_bar.set_postfix(loss=f"{loss.item():.4f}")
    
    avg_val_loss = total_val_loss / len(val_loader)
    print(f"[Epoch {epoch}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    if avg_val_loss < best_val_loss:
        if best_model_path and os.path.exists(best_model_path):
            os.remove(best_model_path)
        best_val_loss = avg_val_loss
        best_model_path = os.path.join(SAVE_DIR, "best.pt")
        torch.save(model.state_dict(), best_model_path)
        print(f"  -> New best model saved at {best_model_path}")


Epoch 1/250


🚀 Training:   0%|          | 0/102 [00:00<?, ?it/s]/home/wicaksonolxn/Documents/learnNLP/3.seq2seqTransformer/src/dataloader/_collate_fn.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_batch = [torch.tensor(item['src'], dtype=torch.long) for item in batch]
/home/wicaksonolxn/Documents/learnNLP/3.seq2seqTransformer/src/dataloader/_collate_fn.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_batch = [torch.tensor(item['tgt'], dtype=torch.long) for item in batch]
🚀 Validation: 100%|██████████| 21/21 [00:00<00:00, 223.45it/s, loss=8.4977]


[Epoch 1] Train Loss: 8.5404 | Val Loss: 8.4775
  -> New best model saved at saved/best.pt
Epoch 2/250


🚀 Validation: 100%|██████████| 21/21 [00:00<00:00, 192.19it/s, loss=8.3446]


[Epoch 2] Train Loss: 8.4069 | Val Loss: 8.3251
  -> New best model saved at saved/best.pt
Epoch 3/250


...

[Epoch 248] Train Loss: 0.8370 | Val Loss: 0.4199
  -> New best model saved at saved/best.pt
Epoch 249/250


🚀 Validation: 100%|██████████| 21/21 [00:00<00:00, 267.02it/s, loss=0.3945]


[Epoch 249] Train Loss: 0.8374 | Val Loss: 0.4160
  -> New best model saved at saved/best.pt
Epoch 250/250


🚀 Validation: 100%|██████████| 21/21 [00:00<00:00, 283.50it/s, loss=0.3908]


[Epoch 250] Train Loss: 0.8273 | Val Loss: 0.4121
  -> New best model saved at saved/best.pt


## Bleu Score

In [ ]:
import os
import torch
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# Load the best model for testing.
encoder = TransformerEncoder(SRC_VOCAB_SIZE, D_MODEL, N_LAYERS, N_HEADS, FFN_HIDDEN, DROPOUT, device)
decoder = TransformerDecoder(TGT_VOCAB_SIZE, D_MODEL, N_LAYERS, N_HEADS, FFN_HIDDEN, DROPOUT, device)
best_model = Transformer(encoder, decoder, device, utils.PAD_TOKEN).to(device)
best_model.load_state_dict(torch.load(os.path.join(SAVE_DIR, "best.pt")))
print("Loaded best model for testing!")

def greedy_decode(model, src, max_len, start_symbol=utils.SOS_TOKEN):
    model.eval()
    src_mask = model.make_input_mask(src)  # shape: [batch, 1, 1, src_len]
    memory = model.encoder(src, src_mask)
    batch_size = src.size(0)
    ys = torch.full((batch_size, max_len), utils.PAD_TOKEN, dtype=torch.long, device=src.device)
    ys[:, 0] = start_symbol
    current_length = 1
    for i in range(1, max_len):
        current_tgt = ys[:, :current_length]
        padded_tgt = torch.cat([
            current_tgt,
            torch.full((batch_size, max_len - current_length), utils.PAD_TOKEN, device=src.device)
        ], dim=1)
        out, _ = model(src, padded_tgt)
        predicted_logits = out[:, current_length - 1, :]  
        next_token = predicted_logits.argmax(dim=-1)
        ys[:, current_length] = next_token
        current_length += 1
        if next_token.item() == utils.EOS_TOKEN:
            break
    return ys[:, :current_length]

# Prepare BLEU evaluation.
smooth_fn = SmoothingFunction().method1
references = []
hypotheses = []
best_model.eval()

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        # Ensure batch dimensions are correct for src and tgt.
        if src_batch.dim() == 1:
            src_batch = src_batch.unsqueeze(0)
        if tgt_batch.dim() == 1:
            tgt_batch = tgt_batch.unsqueeze(0)
            
        preds = greedy_decode(best_model, src_batch, max_len=70)
        
        for i in range(src_batch.size(0)):
            gold = tgt_batch[i].tolist()
            # Remove SOS and PAD tokens from gold.
            gold_tokens = [str(t) for t in gold if t not in [utils.SOS_TOKEN, utils.PAD_TOKEN]]
            if utils.EOS_TOKEN in gold_tokens:
                gold_tokens = gold_tokens[:gold_tokens.index(utils.EOS_TOKEN)]
            pred = preds[i].tolist()
            # Remove SOS from predictions.
            pred_tokens = [str(t) for t in pred if t != utils.SOS_TOKEN]
            if utils.EOS_TOKEN in pred_tokens:
                pred_tokens = pred_tokens[:pred_tokens.index(utils.EOS_TOKEN)]
            references.append([gold_tokens])
            hypotheses.append(pred_tokens)
weights = (0.5, 0.5)
bleu_score = corpus_bleu(
    references,
    hypotheses,
    weights=weights,
    smoothing_function=smooth_fn
)
print(f"Test BLEU-2: {bleu_score:.4f}")


Loaded best model for testing!


Testing: 100%|██████████| 664/664 [01:16<00:00,  8.73it/s]

Test BLEU-2: 0.0000


## Translasi

In [ ]:
import os
import pickle
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# -----------------------------------------------------------------------------
# 1. Load the best model for testing
# -----------------------------------------------------------------------------

# Create encoder, decoder, and the full Transformer model.
encoder = TransformerEncoder(SRC_VOCAB_SIZE, D_MODEL, N_LAYERS, N_HEADS, FFN_HIDDEN, DROPOUT, device)
decoder = TransformerDecoder(TGT_VOCAB_SIZE, D_MODEL, N_LAYERS, N_HEADS, FFN_HIDDEN, DROPOUT, device)
best_model = Transformer(encoder, decoder, device, utils.PAD_TOKEN).to(device)

# Load the best model state
model_path = os.path.join(SAVE_DIR, "best.pt")
best_model.load_state_dict(torch.load(model_path, map_location=device))
print("Loaded best model for testing!")
# -----------------------------------------------------------------------------
# 2. Load the input and output dictionaries
# -----------------------------------------------------------------------------
pth = "dataset"
src = "min"
tgt = "eng"
tp  = os.path.join(pth,f"{src}_{tgt}")
input_dic_path = os.path.join(tp, "input_dic.pkl")
output_dic_path = os.path.join(tp, "output_dic.pkl")
with open(input_dic_path, "rb") as f:
    input_dictionary = pickle.load(f)
with open(output_dic_path, "rb") as f:
    output_dictionary = pickle.load(f)
# -----------------------------------------------------------------------------
# 4. Define a translation function that uses tokenization, model inference, and detokenization
# -----------------------------------------------------------------------------
def translate_sentence(sentence, model, input_dictionary, output_dictionary, max_length=50):
    tokenized = utils.tokenize(sentence, input_dictionary, MAX_LENGTH=max_length)
    src_tensor = torch.tensor(tokenized).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output_tokens = greedy_decode(model, src_tensor, max_length)
    # Convert the output token indices back to words using detokenize
    translated_sentence = utils.detokenize(output_tokens[0].tolist(), output_dictionary)
    return translated_sentence

# -----------------------------------------------------------------------------
# 5. Process a list of sentences and output their translations
# -----------------------------------------------------------------------------
minang_sentences = [
    "Ambo mancari awaknyo besok",
    "Alun salama, apo kabar?",
    "Dunsanak ka rumah gadang",
    "Urang minang manarimo tradisi",
    "Apo ado di pasar?"
]

for sentence in minang_sentences:
    translation = translate_sentence(sentence, best_model, input_dictionary, output_dictionary)
    print(f"Original:   {sentence}")
    print(f"Translated: {translation}\n")


Loaded best model for testing!
Original:   Ambo mancari awaknyo besok
Translated: 

Original:   Alun salama, apo kabar?
Translated: 

Original:   Dunsanak ka rumah gadang
Translated: 

Original:   Urang minang manarimo tradisi
Translated: 

Original:   Apo ado di pasar?
Translated: 



# Translasi: 
Ambo mancari awaknyo besok

    Bahasa Indonesia: Saya akan mencarimu besok.
    English: I will look for you tomorrow.

Alun salama, apo kabar?

    Bahasa Indonesia: Halo, apa kabar?
    English: Hello, how are you?

Dunsanak ka rumah gadang

    Bahasa Indonesia: Saudara, mari ke rumah gadang.
    English: Relatives, let's go to the traditional house.

Urang minang manarimo tradisi

    Bahasa Indonesia: Orang Minang menerima tradisi.
    English: Minangkabau people embrace tradition.

Apo ado di pasar?

    Bahasa Indonesia: Apa ada di pasar?
    English: What's there in the market?